In [1]:
import pandas as pd
import numpy as np
import datetime
from pathlib import Path
import re

#### 파일 로드

In [2]:
def load_file_utf8(path, file_name) :
    file = pd.read_csv('{0}/{1}.csv'.format(path, file_name), encoding='utf-8')
    return file

#### Function 필요 없는 사원명 삭제

In [3]:
def delete(df, path):
    delete_list_df = load_file_utf8(path, delete_list)
    con = df['사원'].isin(delete_list_df['사원'].values.tolist())
    df=df.loc[~con]
    return df

#### Function 계약직(일반사무직) 직원의 직급 수정

In [4]:
def tem_employee(df, path):
    temp_employee_df = load_file_utf8(path, temp_employee_list_file_name)
    con = df['사원'].isin(temp_employee_df['사원'].values.tolist())
    target_index = df.loc[con].loc[:,'직급'].index
    df.loc[target_index, '직급'] = '계약직'
    return df

#### Function 결과표를 직제순서로 정렬

In [5]:
def arrange_department(df, date, path) :
    order_df = load_file_utf8(path, '직제순서')
    order_df['시점']=pd.to_datetime(order_df['시점'], format='%Y-%m-%d')
    order_df=order_df[order_df['시점']==date]    
    df_merge = pd.merge(order_df, df, left_on='부서', right_on=df.index)
    df_merge = df_merge.sort_values(by=['직제순서'])
    df_merge=df_merge.drop(['시점', '직제순서'], axis=1)
    return df_merge

#### -------- 아래부터는 결과물 작성하는 탭 ---------

#### 작업 폴더 경로 정의

In [6]:
project_folder_name = '직원현황작업폴더'
project_folder_path = Path.home().joinpath('Desktop', project_folder_name).absolute()
working_folder_path = project_folder_path.joinpath('202110141545')

print(f'project_folder_path : {project_folder_path}', f'working_folder_path : {working_folder_path}', sep='\n')

project_folder_path : C:\Users\user\Desktop\직원현황작업폴더
working_folder_path : C:\Users\user\Desktop\직원현황작업폴더\202110141545


#### Input 파일명 정의

In [7]:
employee_list_file_name = '사원명부'
delete_list = '지우기목록'
temp_employee_list_file_name = '계약직목록'
order_of_teams = '직제순서'
order_of_positions = '직급순서'

In [8]:
load_file_utf8(working_folder_path, delete_list)  # 파일내용 확인

,사원
0,테스트팀장
1,테스트팀원1
2,테스트팀원2
3,테스트팀원3
4,노조
5,정보관리
6,감사
7,1감사
8,2감사
9,경영지원부


#### Load 사원명부 

In [9]:
df = load_file_utf8(working_folder_path, employee_list_file_name)
# df

#### 사원명명부에서 all null인 행 삭제

In [10]:
con = df.isnull().all(axis=1)
df.loc[con]

,선택,사진,부서,근무부서,사원,사번,주민등록번호M,성별,나이,발령일,...,근무장소,증명서담당자,2015년 건강검진 의무대상 유무,2015년 종합근무성적평가 결과,2016년 종합근무성적평가 결과,2017년 종합근무성적평가 결과,2018년 종합근무성적평가 결과,2019년 종합근무성적평가 결과,2020년 종합근무성적평가 결과,임신여부
206,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df.dropna(how='all', inplace=True)
df.loc[con]

,선택,사진,부서,근무부서,사원,사번,주민등록번호M,성별,나이,발령일,...,근무장소,증명서담당자,2015년 건강검진 의무대상 유무,2015년 종합근무성적평가 결과,2016년 종합근무성적평가 결과,2017년 종합근무성적평가 결과,2018년 종합근무성적평가 결과,2019년 종합근무성적평가 결과,2020년 종합근무성적평가 결과,임신여부


#### 의심되는 사원 목록 확인

In [12]:
con_1 = df['사원'].str.extract(r'^([가-힣]{0,2})$').notnull().squeeze()  # 이름이 2글자 미만인 사원
con_2 = df['사원'].str.extract(r'^(.{4,})$').notnull().squeeze()  # 이름이 네글자 이상인 사원
con_3 = df['사원'].str.extract(r'(\d{1})').notnull().squeeze()  # 숫자가 들어간 사원
con_4 = df['사원'].str.extract(r'(감사)').notnull().squeeze()  # 이름에 감사가 들어간 사원
df.loc[con_1|con_2|con_3|con_4].loc[:,'사원']

0        테스트팀장
1       테스트팀원1
2       테스트팀원2
3       테스트팀원3
4           노조
7         제갈현실
24        정보관리
32          감사
33         1감사
34         2감사
35       경영지원부
45       애니파이브
93        김슬빛나
123    임치시스템관리
173         이전
203         이헌
205        감사실
Name: 사원, dtype: object

#### 지우기 목록 파일에 있는 사원 삭제

In [13]:
df=delete(df, working_folder_path)
df

,선택,사진,부서,근무부서,사원,사번,주민등록번호M,성별,나이,발령일,...,근무장소,증명서담당자,2015년 건강검진 의무대상 유무,2015년 종합근무성적평가 결과,2016년 종합근무성적평가 결과,2017년 종합근무성적평가 결과,2018년 종합근무성적평가 결과,2019년 종합근무성적평가 결과,2020년 종합근무성적평가 결과,임신여부
5,NaN,NaN,대중소기업농어업협력재단,대중소기업농어업협력재단,김순철,201902001.0,600725-1******,남자,62.0,2019-02-01,...,NaN,False,False,NaN,NaN,NaN,0,0,0,False
6,NaN,NaN,기획조정본부,기획조정본부,국신욱,200803001.0,720912-1******,남자,50.0,2019-05-21,...,4층,False,False,B,B(86.46),A(88.55),A(91.49),S(88.25),A(92.39),False
7,NaN,NaN,기획조정본부,기획조정본부,제갈현실,200501003.0,770803-2******,여자,45.0,2021-02-05,...,12층,False,True,B,B(87.04),B(85.85),S(91.24),S(87.49),D(80.54),False
8,NaN,NaN,기획조정본부,기획조정본부,김화영,201204002.0,790828-2******,여자,43.0,2021-10-12,...,8층,False,True,A,A(89.11),B(84.13),C(82.55),B(84.79),B(86.57),False
9,NaN,NaN,기획조정본부,기획운영부,이희영,201007002.0,860213-2******,여자,36.0,2021-06-14,...,4층,False,False,NaN,B(82.46),A(85.41),B(87.08),B(84.88),D(80.64),False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,NaN,NaN,동반성장전략팀,동반성장전략팀,김호영,201006001.0,800821-1******,남자,42.0,2020-03-01,...,8층,False,True,A,S(90.58),S(90.04),B(87.59),B(85.92),A(87.83),False
201,NaN,NaN,동반성장전략팀,동반성장전략팀,이웅희,201310011.0,881213-1******,남자,34.0,2021-01-11,...,12층,False,False,A,B(82.47),B(82.88),S(90.00),B(85.49),B(87.63),False
202,NaN,NaN,동반성장전략팀,농어촌기금관리부,백건하,201703007.0,880517-1******,남자,34.0,2020-03-09,...,NaN,False,False,NaN,NaN,D(78.01),D(81.4),B(82.18),D(79.47),False
203,NaN,NaN,감사실,감사실,이헌,200902002.0,740822-1******,남자,48.0,2021-01-11,...,4층,False,False,B,B(87.38),B(87.01),C(89.47),D(81.64),D(83.08),False


#### 계약직 사원의 직급 명칭 변경

In [14]:
df = tem_employee(df, working_folder_path)
df[df['사원']=='김슬빛나']['직급']

93    계약직
Name: 직급, dtype: object

#### 최종 사원명부 csv파일 저장

In [15]:
timestamp = datetime.datetime.today().strftime('%Y%m%d_%H%M%S')
df.to_csv("{0:s}/{1}_사원명부.csv".format(str(working_folder_path), timestamp),encoding="cp949")

#### 직원 현원표 생성

In [16]:
result_df = pd.crosstab(df['부서'], df['직급'])
result_df

직급,1급,2급,3급,4급,5급,계약직,본부장,사무총장,운영국장,전문사무직 나급,전문사무직 다급,파견5급
부서,,,,,,,,,,,,
감사실,0,1,0,1,0,0,0,0,0,0,0,0
경영지원부,0,0,1,1,4,0,0,0,0,0,0,4
기술보호지원부,1,0,2,5,4,1,0,0,0,0,2,1
기술임치운영부,0,0,2,2,3,0,0,0,0,0,0,2
기획운영부,0,0,1,3,4,0,0,0,0,0,0,1
기획조정본부,1,0,2,3,4,0,0,0,0,0,0,0
농어촌기금관리부,0,1,0,1,4,0,0,0,0,0,0,0
농어촌기금운영부,0,1,2,0,2,0,0,0,0,0,0,2
농어촌상생기금운영본부,0,0,0,0,0,0,1,0,0,0,0,0


#### 직원 현원표에 소계와 총계 산출하고 열 순서 정리

In [17]:
result_df['소계'] = result_df['사무총장']+result_df['운영국장']+result_df['본부장']+result_df['1급']+result_df['2급']+result_df['3급']+result_df['4급']+result_df['5급']
result_df['총계'] = result_df['소계']+result_df['계약직']+result_df['전문사무직 나급']+result_df['전문사무직 다급']+result_df['파견5급']
positions = ['사무총장', '운영국장', '본부장', '1급', '2급', '3급', '4급', '5급', '소계', '계약직',
                 '전문사무직 나급', '전문사무직 다급', '파견5급', '총계']
result_df = result_df[positions]
result_df

직급,사무총장,운영국장,본부장,1급,2급,3급,4급,5급,소계,계약직,전문사무직 나급,전문사무직 다급,파견5급,총계
부서,,,,,,,,,,,,,,
감사실,0,0,0,0,1,0,1,0,2,0,0,0,0,2
경영지원부,0,0,0,0,0,1,1,4,6,0,0,0,4,10
기술보호지원부,0,0,0,1,0,2,5,4,12,1,0,2,1,16
기술임치운영부,0,0,0,0,0,2,2,3,7,0,0,0,2,9
기획운영부,0,0,0,0,0,1,3,4,8,0,0,0,1,9
기획조정본부,0,0,0,1,0,2,3,4,10,0,0,0,0,10
농어촌기금관리부,0,0,0,0,1,0,1,4,6,0,0,0,0,6
농어촌기금운영부,0,0,0,0,1,2,0,2,5,0,0,0,2,7
농어촌상생기금운영본부,0,0,1,0,0,0,0,0,1,0,0,0,0,1


#### 직원 현원표를 직제순서로 정리

In [18]:
date = datetime.datetime(2021,1,1)
result_df = arrange_department(result_df, date, working_folder_path)
result_df

,부서,사무총장,운영국장,본부장,1급,2급,3급,4급,5급,소계,계약직,전문사무직 나급,전문사무직 다급,파견5급,총계
0,대중소기업농어업협력재단,1,0,0,0,0,0,0,0,1,0,0,0,0,1
1,감사실,0,0,0,0,1,0,1,0,2,0,0,0,0,2
2,기획조정본부,0,0,0,1,0,2,3,4,10,0,0,0,0,10
3,기획운영부,0,0,0,0,0,1,3,4,8,0,0,0,1,9
4,경영지원부,0,0,0,0,0,1,1,4,6,0,0,0,4,10
5,상생기금부,0,0,0,0,0,1,1,3,5,1,0,0,1,7
6,상생협력본부,0,0,1,0,0,0,0,0,1,0,0,0,0,1
7,상생정책지원부,0,0,0,0,1,1,1,3,6,0,0,0,0,6
8,협력성과확산부,0,0,0,0,1,0,5,4,10,0,0,0,1,11
9,판로지원부,0,0,0,0,1,0,2,7,10,0,0,0,0,10


#### 최종 현황표 csv파일 저장

In [19]:
timestamp = datetime.datetime.today().strftime('%Y%m%d_%H%M%S')
result_df.to_csv("{0:s}/{1}_직원현원표.csv".format(str(working_folder_path), timestamp),encoding="cp949")

In [20]:
compoDf = df[['부서', '사원', '직급']]

In [21]:
def creatComposition(df, result_df) :
    positions = ['사무총장', '운영국장', '본부장', '1급', '2급', '3급', '4급', '5급', '계약직',
                 '전문사무직 나급', '전문사무직 다급', '파견5급']

    dic = {'사무총장': np.nan, '운영국장':np.nan, '본부장':np.nan, '1급':np.nan, '2급':np.nan, '3급':np.nan, '4급':np.nan, '5급':np.nan, 
           '계약직':np.nan, '전문사무직 나급':np.nan, '전문사무직 다급':np.nan, '파견5급':np.nan}

    depart = df.loc[:, '부서'].unique().tolist()[0]
    con = result_df.loc[:, '부서']==depart
    departNumS = result_df.loc[con]
    
    for posi in positions :
        departNum = departNumS[posi].iat[0]
        if departNum == 0 :
            departNum = '-'
        else :
            departNum = f'({str(departNum)})'
        con = df.loc[:, '직급'] == posi
        lst = [departNum] + df.loc[con].loc[:, '사원'].values.tolist()
        string = "\n".join(lst)
        dic[posi] = string
            
    return dic

In [22]:
lstDepart = result_df['부서'].tolist()

In [23]:
baseDf = pd.DataFrame(columns=['사무총장', '운영국장', '본부장', '1급', '2급', '3급', '4급', '5급', '계약직',
                 '전문사무직 나급', '전문사무직 다급', '파견5급'])

for depart in lstDepart :
    
    con = compoDf['부서']==depart
    testDf = compoDf.loc[con]
    s = creatComposition(testDf, result_df)
    s = pd.Series(s, name=depart)
    baseDf = baseDf.append(s, ignore_index=True)
baseDf.index = lstDepart

In [24]:
baseDf2 = pd.DataFrame(columns=['소계', '총계'])
for depart in lstDepart :
    con = result_df.loc[:, '부서'] == depart
    sum1 = result_df.loc[con].loc[:,'소계'].iat[0]
    sum2 = result_df.loc[con].loc[:,'총계'].iat[0]
    srs = pd.Series([sum1, sum2], index=['소계', '총계'])
    baseDf2 = baseDf2.append(srs, ignore_index=True)
compoDf = pd.concat([baseDf.reset_index(), baseDf2.reset_index(drop=True)], axis=1, join='inner')
compoDf = compoDf.set_index('index')
compoDf.index.name = '부서'
orderedColumns = ['사무총장', '운영국장', '본부장', '1급', '2급', '3급', '4급', '5급', '소계', '계약직', '전문사무직 나급',
       '전문사무직 다급', '파견5급', '총계']
compoDf = compoDf[orderedColumns]
compoDf

,사무총장,운영국장,본부장,1급,2급,3급,4급,5급,소계,계약직,전문사무직 나급,전문사무직 다급,파견5급,총계
부서,,,,,,,,,,,,,,
대중소기업농어업협력재단,(1)\n김순철,-,-,-,-,-,-,-,1,-,-,-,-,1
감사실,-,-,-,-,(1)\n이헌,-,(1)\n정재호,-,2,-,-,-,-,2
기획조정본부,-,-,-,(1)\n국신욱,-,(2)\n제갈현실\n김화영,(3)\n이희영\n송채민\n나정준,(4)\n진재권\n김영지\n이혜옥\n윤진선,10,-,-,-,-,10
기획운영부,-,-,-,-,-,(1)\n강건영,(3)\n추지은\n진성룡\n김동욱,(4)\n권세은\n김영수\n박은별\n조혜인,8,-,-,-,(1)\n정예슬,9
경영지원부,-,-,-,-,-,(1)\n이승훈,(1)\n김광덕,(4)\n김나래\n노기호\n장수웅\n김요한,6,-,-,-,(4)\n엄우현\n전다슬\n나민지\n김성웅,10
상생기금부,-,-,-,-,-,(1)\n김도환,(1)\n이종욱,(3)\n위지원\n백정선\n정주호,5,(1)\n신지원,-,-,(1)\n이은비,7
상생협력본부,-,-,(1)\n이영숙,-,-,-,-,-,1,-,-,-,-,1
상생정책지원부,-,-,-,-,(1)\n이정훈,(1)\n배성환,(1)\n김민성,(3)\n김현범\n권남현\n이선경,6,-,-,-,-,6
협력성과확산부,-,-,-,-,(1)\n이종원,-,(5)\n신용준\n박태우\n박형수\n길명원\n장아름,(4)\n강주영\n이연희\n배수현\n이준모,10,-,-,-,(1)\n김지은,11


In [25]:
timestamp = datetime.datetime.today().strftime('%Y%m%d_%H%M%S')
compoDf.to_csv("{0:s}/{1}_부서별구성원표.csv".format(str(working_folder_path), timestamp),encoding="cp949")

In [67]:
result_df_ = result_df.set_index('부서')

result_df_1 = result_df_.loc['대중소기업농어업협력재단':'농어촌기금관리부']
result_1_sum = result_df_1.apply(lambda x: x.sum(), axis=0)
result_1_sum.name = '재단 합계'

result_df_2 = result_df_.loc['동반위 운영국':]
result_2_sum = result_df_2.apply(lambda x: x.sum(), axis=0)
result_2_sum.name = '위원회 합계'

result_sum = result_df_.apply(lambda x: x.sum(), axis=0)
result_sum.name = '재단-위원회 합계'

compoDf_ = compoDf.append(result_1_sum)
compoDf_ = compoDf_.append(result_2_sum)
compoDf_ = compoDf_.append(result_sum)


foundationIndice = result_df_.loc['대중소기업농어업협력재단':'농어촌기금관리부'].index.tolist()
commiteeIndice = result_df_.loc['동반위 운영국':].index.tolist()

orderedIndice = foundationIndice + ['재단 합계'] + commiteeIndice + ['위원회 합계'] + ['재단-위원회 합계']

compoDf_ = compoDf_.reindex(index=orderedIndice)
compoDf_

,사무총장,운영국장,본부장,1급,2급,3급,4급,5급,소계,계약직,전문사무직 나급,전문사무직 다급,파견5급,총계
부서,,,,,,,,,,,,,,
대중소기업농어업협력재단,(1)\n김순철,-,-,-,-,-,-,-,1,-,-,-,-,1
감사실,-,-,-,-,(1)\n이헌,-,(1)\n정재호,-,2,-,-,-,-,2
기획조정본부,-,-,-,(1)\n국신욱,-,(2)\n제갈현실\n김화영,(3)\n이희영\n송채민\n나정준,(4)\n진재권\n김영지\n이혜옥\n윤진선,10,-,-,-,-,10
기획운영부,-,-,-,-,-,(1)\n강건영,(3)\n추지은\n진성룡\n김동욱,(4)\n권세은\n김영수\n박은별\n조혜인,8,-,-,-,(1)\n정예슬,9
경영지원부,-,-,-,-,-,(1)\n이승훈,(1)\n김광덕,(4)\n김나래\n노기호\n장수웅\n김요한,6,-,-,-,(4)\n엄우현\n전다슬\n나민지\n김성웅,10
상생기금부,-,-,-,-,-,(1)\n김도환,(1)\n이종욱,(3)\n위지원\n백정선\n정주호,5,(1)\n신지원,-,-,(1)\n이은비,7
상생협력본부,-,-,(1)\n이영숙,-,-,-,-,-,1,-,-,-,-,1
상생정책지원부,-,-,-,-,(1)\n이정훈,(1)\n배성환,(1)\n김민성,(3)\n김현범\n권남현\n이선경,6,-,-,-,-,6
협력성과확산부,-,-,-,-,(1)\n이종원,-,(5)\n신용준\n박태우\n박형수\n길명원\n장아름,(4)\n강주영\n이연희\n배수현\n이준모,10,-,-,-,(1)\n김지은,11


In [66]:
timestamp = datetime.datetime.today().strftime('%Y%m%d_%H%M%S')
compoDf_.to_csv("{0:s}/{1}_부서별구성원표.csv".format(str(working_folder_path), timestamp),encoding="cp949")